In [1]:
import cv2
import torch
import numpy as np
from keras.models import load_model # type: ignore
from keras.applications.imagenet_utils import preprocess_input # type: ignore
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:

class_names = [
    "Speed limit 20", "Speed limit 30", "Speed limit 50", "Speed limit 60",
    "Speed limit 70", "Speed limit 80", "End of speed limit 80", "Speed limit 100",
    "Speed limit 120", "No passing", "No passing for trucks", "Right-of-way at intersection",
    "Priority road", "Yield", "Stop", "No vehicles", "Vehicles over 3.5t prohibited",
    "No entry", "General caution", "Dangerous curve left", "Dangerous curve right",
    "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right",
    "Road work", "Traffic signals", "Pedestrians", "Children crossing",
    "Bicycles crossing", "Beware of ice/snow", "Wild animals crossing",
    "End of all restrictions", "Turn right ahead", "Turn left ahead",
    "Ahead only", "Go straight or right", "Go straight or left", "Keep right",
    "Keep left", "Roundabout", "End of no passing", "End of no passing for trucks"
]

def preprocess_cnn(image):
    image = cv2.resize(image, (32, 32))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov5_fine_tuned_v01.pt")
cnn_model = load_model('traffic_sign_model_v01.h5')

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    result = results[0]

    boxes = result.boxes

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cropped_img = frame[y1:y2, x1:x2]

        preds = cnn_model.predict(preprocess_cnn(cropped_img))
        pred_class = class_names[np.argmax(preds)]
        print(f"Detected class: {pred_class}")

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{pred_class}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)


    annotated_frame = results[0].plot()

    cv2.imshow("Traffic Sign Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
from ultralytics import YOLO

model = YOLO("yolov8_fine_tuned_v04.pt")

In [17]:

image_folder = "GTSDB/images/train"
labels_folder = "GTSDB/labels/train"

output_file = "v4_train_pred.txt"

# Open output file for writing
with open(output_file, "w") as f:
    # Loop over each image
    for img_name in os.listdir(image_folder):
        if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(image_folder, img_name)
            results = model.predict(source=img_path, save=False, conf=0.25, verbose=False)

            # Get class IDs for detections in current image
            class_ids = results[0].boxes.cls.tolist()

            # Write to file: image_name class_id class_id ...
            f.write(f"{img_name} " + " ".join(str(int(cls_id)) for cls_id in class_ids) + "\n")


In [18]:
with open(output_file, "r") as f:
    lines = f.readlines()

In [19]:
pred = []
for line in lines:
    parts = line.strip().split(' ')
    img_name = parts[0]
    if len(parts) > 1:
        class_ids = [int(cls_id) for cls_id in parts[1:]]
    else:
        class_ids = [99]
    if class_ids:
        pred.append((img_name, class_ids))

In [20]:
actual = []
for file in os.listdir(labels_folder):
    with open(os.path.join(labels_folder, file), "r") as f:
        content = f.readlines()
        content = [line.strip() for line in content if line.strip()]
        if content:
            class_id = []
            for line in content:
                class_id.append(int(line.split()[0]))
            actual.append((file, class_id))   

In [21]:
sum = 0
count = 0

for i in range(len(actual)):
    count += (max(len(actual[i][1]), len(pred[i][1])))

for i in range(len(actual)):
    for j in pred[i][1]:
        if j in actual[i][1]:
            sum += 1
accuracy = sum / count
accuracy

0.9193934557063048